## Support Vector Machine Model

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import scipy.stats as ss

# Make plots larger
plt.rcParams['figure.figsize'] = (15, 9)

In [2]:
AAPL = pd.read_csv('AAPL_New.csv')
AAPL.set_index('Date')
AAPL['OpenTmr'] = AAPL['Open'].shift(-1)
AAPL['OpenClose'] = (AAPL['Open']+ AAPL['Close'])/2
AAPL['HighLow'] = (AAPL['High']+ AAPL['Low'])/2
AAPL['OCHL'] = (AAPL['Open']+ AAPL['Close']+AAPL['High']+ AAPL['Low'])/4
# AAPL = AAPL.fillna(method='ffill')
AAPL.tail()
# AAPL = AAPL.dropna(0)

,Date,High,Low,Open,Close,Volume,Adj Close,OpenTmr,OpenClose,HighLow,OCHL
983,11/27/2019,267.980011,265.309998,265.579987,267.839996,16308900,267.839996,266.600006,266.709991,266.645004,266.677498
984,11/29/2019,268.000000,265.899994,266.600006,267.250000,11654400,267.250000,267.269989,266.925003,266.949997,266.937500
985,12/2/2019,268.250000,263.450012,267.269989,264.160004,23621800,264.160004,258.309998,265.714996,265.850006,265.782501
986,12/3/2019,259.529999,256.290008,258.309998,259.450012,28607600,259.450012,261.070007,258.880005,257.910004,258.395004
987,12/4/2019,263.309998,260.679993,261.070007,261.739990,15874176,261.739990,NaN,261.404999,261.994995,261.699997


In [3]:
N = AAPL.shape[0] # total num days
num_boot = 1# total num bootstrap
T= 250 # start day
window = 200 # training period window
Y =AAPL['OpenTmr'][-(N-(T+1)):].values

In [4]:
from sklearn.svm import SVR  


def svm_train(X_train, Y_train, X_pred):  
   
    svr_model = SVR(kernel='rbf', gamma=0.0005)
    result = svr_model.fit(X_train, Y_train)
 
    y_hat = result.predict(X_train)  
    y_pred = result.predict(X_pred)
    rmse_svm=np.sqrt(np.mean((Y_train-y_hat)**2))
    return y_hat, y_pred, rmse_svm

In [5]:
def svm_bstr():
    num_boot = 1
    yhat_svm = np.zeros(N-(T+1))
    for t in range(T+1, N):    
        X_train = AAPL[['Close']][t-window:t-1] 
        X_train = np.column_stack([np.ones((len(X_train),1)),X_train])
        X_pred = AAPL[['Close']][t-1:t]
        X_pred = np.column_stack([np.ones((len(X_pred),1)),X_pred])
        
        Y_train = AAPL['OpenTmr'][t-window:t-1].values    
        yhat_train = svm_train(X_train, Y_train, X_pred)[0] 
        res_train =  Y_train - yhat_train        
        y_pred_all = np.zeros(num_boot)
        # bootstrap method: switching residuals
        for i in range(0, num_boot):
            err = np.random.choice(res_train, (window-1, ), replace=True)
            y_bstr = yhat_train + err                        
            y_pred_bstr = svm_train(X_train, y_bstr, X_pred)[1]
            y_pred_all[i]=y_pred_bstr
          
        y_pred_svm = y_pred_all.mean() # mean of all bootstrap predictions
        yhat_svm[t-(T+1)]=y_pred_svm # do this for each time step        
    rmse_svm=np.sqrt(np.mean((Y[:-1]-yhat_svm[:-1])**2))
    return yhat_svm, rmse_svm

In [7]:
rmse = svm_bstr()[1]

In [ ]:
rmse